In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = "mcts7500_pool.csv" #o your actual file path
df = pd.read_csv(file_path, delimiter=",")  # Adjust delimiter if needed

# Convert 'board_moves' from string to list
df['board_moves'] = df['board_moves'].apply(lambda x: np.array(eval(x)).reshape(6,7))

# Create two-channel input (current player & opponent)
def split_channels(board):
    channel_1 = (board == 1).astype(np.float32)  # Current player's pieces
    channel_2 = (board == -1).astype(np.float32) # Opponent's pieces
    return np.stack([channel_1, channel_2], axis=-1)  # Shape (6,7,2)

X = np.stack(df['board_moves'].apply(split_channels).values)

# One-hot encode target column (play_y)
y = to_categorical(df['play_y'], num_classes=7)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to verify
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (212496, 6, 7, 2), y_train shape: (212496, 7)
X_test shape: (53124, 6, 7, 2), y_test shape: (53124, 7)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters):
    """Creates a deeper residual block with SE block."""
    res = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    res = layers.BatchNormalization()(res)
    res = layers.Conv2D(filters, (3, 3), padding="same")(res)
    res = layers.BatchNormalization()(res)

    # SE Block
    squeeze = layers.GlobalAveragePooling2D()(res)
    squeeze = layers.Dense(filters // 16, activation='relu')(squeeze)
    squeeze = layers.Dense(filters, activation='sigmoid')(squeeze)
    squeeze = layers.Reshape((1, 1, filters))(squeeze)
    res = layers.Multiply()([res, squeeze])

    x = layers.Add()([x, res])
    x = layers.Activation("relu")(x)
    return x

def build_large_resnet():
    inputs = layers.Input(shape=(6, 7, 2))  # 6x7 board with 2 channels

    # Initial Conv Layer
    x = layers.Conv2D(128, (3, 3), padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)

    # Residual Blocks (Increased number of layers)
    for _ in range(10):
        x = residual_block(x, 128)

    # Additional Conv Layer for deeper feature extraction
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully Connected Layers
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(256, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(7, activation="softmax")(x)  # 7 possible moves

    model = models.Model(inputs, outputs)
    return model

# Create the model
model = build_large_resnet()

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Print summary to verify parameter count
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 6, 7, 2)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 6, 7, 128)      │          2,432 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 6, 7, 128)      │            512 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 6, 7, 128)      │        147,584 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 6, 7, 128)      │            512 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 6, 7, 128)      │        147,584 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 6, 7, 128)      │            512 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 128)            │              0 │ batch_normalization_2… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 8)              │          1,032 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │          1,152 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 1, 128)      │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 6, 7, 128)      │              0 │ batch_normalization_2… │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 6, 7, 128)      │              0 │ batch_normalization[0… │
│                           │                        │                │ multiply[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 6, 7, 128)      │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 6, 7, 128)      │        147,584 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 6, 7, 128)      │            512 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 4,802,263 (18.32 MB)

 Trainable params: 4,792,279 (18.28 MB)

 Non-trainable params: 9,984 (39.00 KB)

In [ ]:
# Set training parameters
batch_size = 128
epochs = 15  # Adjust based on performance


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)


Epoch 1/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 83s 24ms/step - accuracy: 0.3293 - loss: 1.8222 - val_accuracy: 0.5215 - val_loss: 1.2554
Epoch 2/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5624 - loss: 1.1332 - val_accuracy: 0.6156 - val_loss: 0.9761
Epoch 3/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.6343 - loss: 0.9382 - val_accuracy: 0.6522 - val_loss: 0.8769
Epoch 4/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.6733 - loss: 0.8345 - val_accuracy: 0.6760 - val_loss: 0.8239
Epoch 5/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.7005 - loss: 0.7640 - val_accuracy: 0.6853 - val_loss: 0.8078
Epoch 6/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.7285 - loss: 0.6987 - val_accuracy: 0.6933 - val_loss: 0.7738
Epoch 7/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.7457 - loss: 0.6528 - val_accuracy: 0.6980 - val_loss: 0.7860
Epoch 8/15
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.7664 -

In [ ]:
# Save the trained model as an .h5 file
model.save("connect4_cnn_model.h5")

In [ ]:
from google.colab import files
files.download("connect4_cnn_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Play Against Bot

In [ ]:
def check_winner(board, player):
    """Checks if the given player has won the game."""
    for row in range(6):
        for col in range(7):
            if col + 3 < 7 and np.all(board[row, col:col+4] == player):  # Horizontal
                return True
            if row + 3 < 6 and np.all(board[row:row+4, col] == player):  # Vertical
                return True
            if row + 3 < 6 and col + 3 < 7 and np.all([board[row+i, col+i] == player for i in range(4)]):  # Diagonal \
                return True
            if row + 3 < 6 and col - 3 >= 0 and np.all([board[row+i, col-i] == player for i in range(4)]):  # Diagonal /
                return True
    return False

def play_connect4(model):
    board = np.zeros((6, 7), dtype=int)
    while True:
        print(board)
        col = int(input("Enter your move (0-6): "))
        if col < 0 or col >= 7 or board[0, col] != 0:
            print("Invalid move. Try again.")
            continue

        # Drop piece for human (player 1)
        row = np.max(np.where(board[:, col] == 0))
        board[row, col] = 1

        # Check if human won
        if check_winner(board, 1):
            print("Congratulations! You win!")
            print(board)
            break

        # Check if board is full
        if np.all(board != 0):
            print("It's a draw!")
            break

        # CNN's move
        input_board = np.stack([(board == 1).astype(np.float32), (board == -1).astype(np.float32)], axis=-1)
        input_board = np.expand_dims(input_board, axis=0)
        ai_move = np.argmax(model.predict(input_board))

        # Drop piece for AI (player -1)
        row = np.max(np.where(board[:, ai_move] == 0))
        board[row, ai_move] = -1

        # Check if AI won
        if check_winner(board, -1):
            print("AI wins! Better luck next time.")
            print(board)
            break

        # Check if board is full
        if np.all(board != 0):
            print("It's a draw!")
            break


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("connect4_model.h5")


In [ ]:
play_connect4(model)


NameError: name 'np' is not defined